In [ ]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Configurazione TPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # Verifica se TPU è disponibile
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strategy = tf.distribute.TPUStrategy(tpu)
    print("TPU rilevata e configurata correttamente!")
except ValueError:
    tpu_strategy = tf.distribute.get_strategy()  # Usa CPU/GPU come fallback
    print("TPU non rilevata. Utilizzo CPU/GPU.")

# Percorsi al dataset
images_path = "/kaggle/input/kvasir-dataset-for-classification-and-segmentation/kvasir-seg/Kvasir-SEG/images"
masks_path = "/kaggle/input/kvasir-dataset-for-classification-and-segmentation/kvasir-seg/Kvasir-SEG/masks"

# Parametri globali
IMG_HEIGHT, IMG_WIDTH = 256, 256
BATCH_SIZE = 128
EPOCHS = 100

# Funzione per il caricamento delle immagini
def load_images_and_masks(images_path, masks_path):
    images = []
    masks = []
    for img_name in os.listdir(images_path):
        img = tf.keras.preprocessing.image.load_img(
            os.path.join(images_path, img_name), target_size=(IMG_HEIGHT, IMG_WIDTH)
        )
        img = tf.keras.preprocessing.image.img_to_array(img) / 255.0
        images.append(img)
        
        mask_name = img_name  # Supponiamo che i file immagine e maschera abbiano lo stesso nome
        mask = tf.keras.preprocessing.image.load_img(
            os.path.join(masks_path, mask_name), color_mode="grayscale", target_size=(IMG_HEIGHT, IMG_WIDTH)
        )
        mask = tf.keras.preprocessing.image.img_to_array(mask) / 255.0
        masks.append(mask)
    
    return np.array(images), np.array(masks)

# Caricamento delle immagini e maschere
images, masks = load_images_and_masks(images_path, masks_path)

# Divisione del dataset in training e validation
X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)

# Costruzione del modello U-Net
def unet_model(input_size=(IMG_HEIGHT, IMG_WIDTH, 3)):
    inputs = Input(input_size)
    
    # Encoder
    c1 = Conv2D(64, (3, 3), activation="relu", padding="same")(inputs)
    c1 = Conv2D(64, (3, 3), activation="relu", padding="same")(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(128, (3, 3), activation="relu", padding="same")(p1)
    c2 = Conv2D(128, (3, 3), activation="relu", padding="same")(c2)
    p2 = MaxPooling2D((2, 2))(c2)
    
    c3 = Conv2D(256, (3, 3), activation="relu", padding="same")(p2)
    c3 = Conv2D(256, (3, 3), activation="relu", padding="same")(c3)
    p3 = MaxPooling2D((2, 2))(c3)
    
    c4 = Conv2D(512, (3, 3), activation="relu", padding="same")(p3)
    c4 = Conv2D(512, (3, 3), activation="relu", padding="same")(c4)
    p4 = MaxPooling2D((2, 2))(c4)
    
    # Bottleneck
    c5 = Conv2D(1024, (3, 3), activation="relu", padding="same")(p4)
    c5 = Conv2D(1024, (3, 3), activation="relu", padding="same")(c5)
    
    # Decoder
    u6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding="same")(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3, 3), activation="relu", padding="same")(u6)
    c6 = Conv2D(512, (3, 3), activation="relu", padding="same")(c6)
    
    u7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding="same")(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3, 3), activation="relu", padding="same")(u7)
    c7 = Conv2D(256, (3, 3), activation="relu", padding="same")(c7)
    
    u8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding="same")(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3, 3), activation="relu", padding="same")(u8)
    c8 = Conv2D(128, (3, 3), activation="relu", padding="same")(c8)
    
    u9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding="same")(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, (3, 3), activation="relu", padding="same")(u9)
    c9 = Conv2D(64, (3, 3), activation="relu", padding="same")(c9)
    
    outputs = Conv2D(1, (1, 1), activation="sigmoid")(c9)
    
    return Model(inputs=[inputs], outputs=[outputs])

# Compilazione e addestramento del modello
with tpu_strategy.scope():
    model = unet_model()
    model.compile(optimizer=Adam(learning_rate=1e-4), loss="binary_crossentropy", metrics=["accuracy"])
    
    # Addestramento del modello
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS
    )
    
    # Salvataggio del modello
    model.save('/kaggle/working/trained_model.keras')  # Salva il modello nella directory di lavoro

# Funzione per calcolare IoU
def iou_score(y_true, y_pred, smooth=1e-7):
    intersection = np.sum(y_true * y_pred)
    union = np.sum(y_true) + np.sum(y_pred) - intersection
    return (intersection + smooth) / (union + smooth)

# Funzione per calcolare DICE coefficient
def dice_coefficient(y_true, y_pred, smooth=1e-7):
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection + smooth) / (np.sum(y_true) + np.sum(y_pred) + smooth)

# Funzione per calcolare le metriche su un dataset
def evaluate_metrics(model, X, y, threshold=0.5):
    predictions = model.predict(X)
    predictions = (predictions > threshold).astype(np.float32)  # Binarizzazione delle previsioni
    
    iou_scores = []
    dice_scores = []
    
    for true, pred in zip(y, predictions):
        iou_scores.append(iou_score(true, pred))
        dice_scores.append(dice_coefficient(true, pred))
    
    mean_iou = np.mean(iou_scores)
    mean_dice = np.mean(dice_scores)
    
    return mean_iou, mean_dice, predictions

# Valutazione sulle immagini di validazione
mean_iou, mean_dice, val_predictions = evaluate_metrics(model, X_val, y_val)

# Risultati numerici
print(f"Mean IoU on Validation Set: {mean_iou:.4f}")
print(f"Mean DICE Coefficient on Validation Set: {mean_dice:.4f}")

# Visualizzazione grafica di IoU e DICE per alcune immagini
def plot_metrics(y_true, y_pred, predictions, num_samples=5):
    indices = np.random.choice(len(y_true), num_samples, replace=False)
    plt.figure(figsize=(15, 15))
    
    for i, idx in enumerate(indices):
        plt.subplot(num_samples, 4, i * 4 + 1)
        plt.imshow(y_true[idx].squeeze(), cmap='gray')
        plt.title("True Mask")
        plt.axis("off")
        
        plt.subplot(num_samples, 4, i * 4 + 2)
        plt.imshow(predictions[idx].squeeze(), cmap='gray')
        plt.title("Predicted Mask")
        plt.axis("off")
        
        plt.subplot(num_samples, 4, i * 4 + 3)
        plt.imshow(y_true[idx].squeeze(), cmap='gray')
        plt.imshow(predictions[idx].squeeze(), cmap='jet', alpha=0.5)
        plt.title("Overlay")
        plt.axis("off")
        
        # IoU e DICE per questa immagine
        img_iou = iou_score(y_true[idx], y_pred[idx])
        img_dice = dice_coefficient(y_true[idx], y_pred[idx])
        
        plt.subplot(num_samples, 4, i * 4 + 4)
        plt.bar(["IoU", "DICE"], [img_iou, img_dice])
        plt.title("Metrics")
        plt.ylim(0, 1)
    
    plt.tight_layout()
    plt.show()

# Visualizzazione delle metriche
plot_metrics(y_val, val_predictions, val_predictions)


In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from PIL import Image
from tensorflow.keras.models import load_model

# Impostazioni delle dimensioni delle immagini
IMG_HEIGHT, IMG_WIDTH = 256, 256  # Dimensioni di input per il modello

# Percorso delle nuove immagini che vuoi testare
new_images_path = "/kaggle/input/polipi-di-verifica"  # Modifica questo percorso con quello giusto

# Caricamento del modello addestrato
model_path = '/kaggle/working/trained_model.keras'  # Percorso del modello salvato
model = tf.keras.models.load_model(model_path)  # Carica il modello


# Funzione per caricare nuove immagini
def load_new_images(images_path):
    new_images = []
    image_names = []
    for img_name in os.listdir(images_path):
        img_path = os.path.join(images_path, img_name)
        try:
            # Carica l'immagine e la ridimensiona
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
            img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0  # Normalizza l'immagine
            new_images.append(img_array)
            image_names.append(img_name)
        except (OSError, Image.UnidentifiedImageError) as e:
            print(f"Warning: Impossibile caricare l'immagine {img_name}. Errore: {e}")
            continue  # Ignora il file non valido
    return np.array(new_images), image_names

# Funzione per visualizzare i risultati delle previsioni
def display_predictions(new_images, predictions, image_names):
    plt.figure(figsize=(15, 15))
    for i in range(len(new_images)):
        # Visualizza l'immagine di input
        plt.subplot(len(new_images), 3, i * 3 + 1)
        plt.imshow(new_images[i])
        plt.title(f"Input Image: {image_names[i]}")
        plt.axis("off")
        
        # Visualizza la maschera predetta
        plt.subplot(len(new_images), 3, i * 3 + 2)
        plt.imshow(predictions[i].squeeze(), cmap='gray')  # Usa 'squeeze' per rimuovere il canale extra
        plt.title("Predicted Mask")
        plt.axis("off")
        
        # Sovrapposizione tra immagine e maschera predetta
        plt.subplot(len(new_images), 3, i * 3 + 3)
        plt.imshow(new_images[i])
        plt.imshow(predictions[i].squeeze(), cmap='jet', alpha=0.5)  # Sovrapposizione in modalità trasparente
        plt.title("Overlay")
        plt.axis("off")
    
    plt.tight_layout()
    plt.show()

# Carica le nuove immagini per la previsione
new_images, new_image_names = load_new_images(new_images_path)

# Verifica se sono state caricate immagini
if new_images.shape[0] == 0:
    print("Nessuna immagine valida trovata nella directory.")
else:
    # Esegui le previsioni sulle nuove immagini
    predictions = model.predict(new_images)

    # Visualizza i risultati
    display_predictions(new_images, predictions, new_image_names)
